# [1단계] 프로젝트 환경 설정 및 데이터 준비

## 1-1. 필요한 라이브러리 설치

In [1]:
# - datasets: 허깅페이스 데이터셋을 쉽게 다루게 해주는 라이브러리
# - openai: OpenAI API를 사용하기 위한 라이브러리
# - PyMuPDF: PDF 파일을 열고 텍스트를 추출하는 데 사용 (fitz라는 이름으로 임포트)
# - pandas: 결과를 표 형태로 다루기 위한 라이브러리
# - tqdm: 오래 걸리는 작업의 진행 상황을 시각적으로 보여주는 막대 바
# - tiktoken: 텍스트를 토큰 단위로 나누고 길이를 계산하는 데 사용 (GPT 모델용)
!pip install -qU datasets openai PyMuPDF "pandas==2.2.2" tqdm tiktoken pdfplumber

## 1-2. 필요한 라이브러리 임포트

In [2]:
import os                     # 운영체제 관련 기능을 사용하기 위함 (여기서는 크게 사용되진 않아)
import json                   # JSON 형식의 데이터를 다루기 위함 (GPT 응답이 JSON이야)
import fitz                   # PyMuPDF 라이브러리. PDF를 열고 텍스트를 추출할 때 사용해
import pandas as pd           # 데이터를 표(DataFrame) 형태로 다루기 위한 라이브러리. pd라는 별명으로 불러
import tiktoken               # OpenAI의 토크나이저. 텍스트를 토큰 단위로 쪼갤 때 사용
from tqdm.notebook import tqdm # 반복문의 진행상황을 예쁜 바로 보여주는 라이브러리
from datasets import load_dataset # 허깅페이스 데이터셋을 로드하는 함수
from openai import OpenAI     # OpenAI API와 통신하기 위한 메인 클라이언트
from google.colab import userdata # Colab의 시크릿(API 키 등)을 안전하게 가져오는 기능

print("✅ 라이브러리 임포트 완료!")

✅ 라이브러리 임포트 완료!


## 1-3. OpenAI API 키 설정 및 클라이언트 초기화

In [3]:
# Colab의 '비밀' 탭에 저장해 둔 OPENAI_API_KEY를 안전하게 불러와.
# 이 키를 사용해서 OpenAI 서비스에 접속할 수 있는 'client' 객체를 만들어.
# 앞으로 우리는 이 client를 통해서 GPT 모델에게 말을 걸게 될 거야.
try:
    # userdata.get('키_이름') 으로 저장된 키 값을 가져온다.
    api_key = userdata.get('OPENAI_API_KEY')

    # 가져온 API 키로 OpenAI 클라이언트를 초기화(생성)한다.
    client = OpenAI(api_key=api_key)

    # 키가 정상적으로 로드되었는지 확인 메시지를 출력한다.
    print("✅ OpenAI API 키가 성공적으로 로드되었습니다.")

except Exception as e:
    # 만약 키를 가져오는 데 실패하면, 사용자에게 알려준다.
    print(f"🚨 OpenAI API 키 로드에 실패했습니다. Colab '비밀' 탭에 'OPENAI_API_KEY'가 올바르게 설정되었는지 확인해주세요.")
    print(f"오류: {e}")

✅ OpenAI API 키가 성공적으로 로드되었습니다.


# [2단계] Hugging Face에서 원본 PDF 데이터셋 로드

## 2-1. 데이터셋 불러오기

In [4]:
# 우리가 사용할 PDF 파일들이 모여있는 데이터셋을 허깅페이스 Hub에서 직접 로드해.
# 데이터셋 주소: https://huggingface.co/datasets/sumilee/SKN14-Final-3Team-Data2
# split='train'은 데이터셋의 'train' 부분만 가져오겠다는 의미야.
try:
    dataset = load_dataset("sumilee/SKN14-Final-3Team-Data2", split='train')

    # 데이터셋 로드가 성공하면 확인 메시지를 출력해.
    print("✅ 데이터셋 로드 성공!")
    print("\n--- 데이터셋 정보 ---")

    # 로드된 데이터셋의 구조를 보여줘. (features, num_rows 등)
    print(dataset)

    # 데이터셋에 총 몇 개의 항목(PDF 파일)이 있는지 길이를 확인해서 출력해.
    print(f"\n총 {len(dataset)}개의 PDF 파일을 로드했습니다.")

except Exception as e:
    # 데이터셋 로드 중 문제가 생기면 오류 메시지를 출력해.
    print(f"🚨 데이터셋 로드 중 오류가 발생했습니다: {e}")
    print("데이터셋 이름, 접근 권한 등을 확인해주세요.")

Resolving data files:   0%|          | 0/233 [00:00<?, ?it/s]

✅ 데이터셋 로드 성공!

--- 데이터셋 정보 ---
Dataset({
    features: ['pdf', 'label'],
    num_rows: 233
})

총 233개의 PDF 파일을 로드했습니다.


# [3단계] PDF 처리 및 Q&A 생성을 위한 핵심 함수 정의

## 3-1. PDF의 바이트 데이터에서 텍스트를 추출하는 함수

In [13]:
def extract_text_from_pdf_bytes(pdf_bytes):
    """
    PDF 파일의 바이트 데이터를 입력받아 PyMuPDF(fitz)를 사용해 전체 텍스트를 추출한다.
    Args:
        pdf_bytes (bytes): PDF 파일의 내용.
    Returns:
        str: 추출된 전체 텍스트. 오류 발생 시 빈 문자열을 반환한다.
    """
    try:
        # 바이트 데이터로부터 PDF 문서를 열어.
        doc = fitz.open(stream=pdf_bytes, filetype="pdf")

        # 각 페이지를 순회하면서 텍스트를 추출하고, page_texts 리스트에 저장해.
        page_texts = [page.get_text() for page in doc]

        # 문서를 닫아서 메모리를 확보해.
        doc.close()

        # 모든 페이지의 텍스트를 하나의 문자열로 합쳐서 반환해.
        return "\n".join(page_texts)

    except Exception as e:
        # 텍스트 추출 중 어떤 오류라도 발생하면, 오류 메시지를 출력하고 빈 문자열을 반환.
        print(f"❗️ PDF 텍스트 추출 중 오류 발생: {e}")
        return ""

## 3-2. 텍스트를 의미있는 조각(청크)으로 나누는 함수

In [14]:
def chunk_text(text, chunk_size=2000, chunk_overlap=200):
    """
    긴 텍스트를 GPT 모델이 처리하기 좋은 크기의 청크로 나눈다.
    토큰 수를 기준으로 자르기 때문에 더 정확하다.
    Args:
        text (str): 나눌 긴 텍스트.
        chunk_size (int): 한 청크의 최대 토큰 수.
        chunk_overlap (int): 청크 간 겹치게 할 토큰 수 (문맥 유지를 위함).
    Returns:
        list: 텍스트 청크들의 리스트.
    """
    try:
        # GPT-4o, GPT-4, gpt-3.5-turbo에서 사용하는 'cl100k_base' 인코더를 가져온다.
        tokenizer = tiktoken.get_encoding("cl100k_base")
        tokens = tokenizer.encode(text)

        chunks = []
        # 반복문을 통해 (chunk_size - chunk_overlap) 만큼 건너뛰면서 토큰을 자른다.
        for i in range(0, len(tokens), chunk_size - chunk_overlap):
            chunk_tokens = tokens[i:i + chunk_size]
            # 토큰을 다시 텍스트로 변환(디코딩)해서 리스트에 추가한다.
            chunk_text = tokenizer.decode(chunk_tokens)
            chunks.append(chunk_text)
        return chunks
    except Exception as e:
        print(f"❗️ 텍스트 청킹 중 오류 발생: {e}")
        return [text] # 청킹 실패 시 원본 텍스트를 리스트에 담아 반환

## 3-3. OpenAI API를 호출하여 Q&A 쌍을 생성하는 함수

In [15]:
def generate_qa_with_openai(text_chunk):
    """
    주어진 텍스트 조각(chunk)을 가지고 OpenAI API를 호출하여 Q&A 쌍을 생성한다.
    Args:
        text_chunk (str): Q&A 생성을 위한 바탕이 될 텍스트.
    Returns:
        str: GPT 모델이 생성한 JSON 형식의 Q&A 문자열. 실패 시 None을 반환한다.
    """
    # GPT에게 역할을 부여하고, 결과물의 형식을 지정하는 시스템 프롬프트.
    system_prompt = """
    You are an AI assistant and a financial expert at KB Financial Group. Your task is to generate a question-and-answer (Q&A) pair based on the provided financial document.
    You must strictly adhere to the following rules:
    1. The "question" must be professional and specific, as if asked by a KB employee or a knowledgeable financial customer.
    2. The "answer" must be based *only* on the provided Context. Do not use any of your prior knowledge.
    3. The final output must be a single JSON object with exactly two keys: "question" and "answer".
    4. Both the question and the answer must be written entirely in Korean.
    """

    try:
        # client.chat.completions.create를 통해 API에 요청을 보낸다.
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 비용 효율적인 최신 모델 사용
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Context:\n---\n{text_chunk}\n---\n위 Context를 바탕으로 JSON 형식의 Q&A 쌍을 생성해 주세요."}
            ],
            response_format={"type": "json_object"}, # 결과물이 항상 JSON 형식을 따르도록 강제
            temperature=0.4, # 약간의 창의성을 부여하되, 사실 기반을 유지
            max_tokens=1024  # 답변이 너무 길어지지 않도록 제한
        )
        # 응답에서 실제 내용(content)만 추출하여 반환한다.
        return response.choices[0].message.content

    except Exception as e:
        # API 호출 중 오류가 발생하면 메시지를 출력하고 None을 반환한다.
        print(f"❗️ OpenAI API 호출 중 오류 발생: {e}")
        return None

## 3-4. GPT의 응답(JSON 문자열)을 파싱하는 함수

In [16]:
def parse_gpt_response(response_str):
    """
    GPT가 생성한 JSON 형식의 문자열을 파싱하여 질문과 답변으로 분리한다.
    Args:
        response_str (str): GPT의 응답 문자열.
    Returns:
        tuple: (question, answer) 튜플. 파싱 실패 시 (None, None)을 반환한다.
    """
    try:
        # JSON 문자열을 파이썬 딕셔너리로 변환한다.
        qa_pair = json.loads(response_str)

        # 딕셔너리에서 'question'과 'answer' 키의 값을 가져온다.
        question = qa_pair.get("question")
        answer = qa_pair.get("answer")

        # 질문과 답변이 모두 존재할 경우에만 값을 반환한다.
        if question and answer:
            return question, answer
        else:
            # 둘 중 하나라도 없으면 실패로 간주한다.
            print(f"❗️ 파싱 실패: 'question' 또는 'answer' 키가 결과에 없습니다. 응답: {response_str}")
            return None, None

    except json.JSONDecodeError:
        # JSON 형식이 잘못되어 파싱에 실패한 경우
        print(f"❗️ JSON 파싱 오류. 응답: {response_str}")
        return None, None
    except Exception as e:
        # 그 외 예상치 못한 오류가 발생한 경우
        print(f"❗️ 응답 파싱 중 예상치 못한 오류 발생: {e}")
        return None, None

print("✅ 4개의 핵심 함수(추출, 청킹, 생성, 파싱)가 성공적으로 정의되었습니다.")

✅ 4개의 핵심 함수(추출, 청킹, 생성, 파싱)가 성공적으로 정의되었습니다.


# [4단계] 모든 PDF를 순회하며 Q&A 데이터 생성 실행

In [20]:
# 최종 Q&A 쌍들을 저장할 빈 리스트를 생성.
all_qa_pairs = []

# 처리할 PDF의 개수를 설정.
NUM_FILES_TO_PROCESS = len(dataset)
# NUM_FILES_TO_PROCESS = 5 # 테스트용 코드: 5개 파일만 먼저 돌려보자

print(f"🚀 총 {NUM_FILES_TO_PROCESS}개의 PDF 파일에 대한 Q&A 생성을 시작합니다...")
print("⚠️ 이 작업은 시간이 오래 걸릴 수 있습니다.")

# tqdm의 바깥쪽 루프는 PDF 파일을 순회한다. (총 233번)
for i in tqdm(range(NUM_FILES_TO_PROCESS), desc="전체 PDF 진행률"):
    item = dataset[i]
    pdf_object = item['pdf']
    pdf_filename = f"pdf_{i}_label_{item['label']}.pdf"

    try:
        pdf_object.stream.seek(0)
        pdf_bytes = pdf_object.stream.read()

        # 3-1 함수: PDF에서 전체 텍스트 추출
        full_text = extract_text_from_pdf_bytes(pdf_bytes)

        if not full_text:
            print(f"❗️ {pdf_filename} 에서 텍스트를 추출하지 못했습니다. (건너뜁니다)")
            continue

        # 3-2 함수: 추출된 전체 텍스트를 여러 개의 청크로 나눔
        text_chunks = chunk_text(full_text, chunk_size=2000, chunk_overlap=200)

        # --- 💡 핵심 수정: 안쪽 루프 추가 💡 ---
        # 이제 텍스트 청크들을 순회하면서 각 청크마다 API를 호출한다.
        for chunk in text_chunks:
            # 3-3 함수: OpenAI API 호출
            gpt_response = generate_qa_with_openai(chunk)

            if gpt_response:
                # 3-4 함수: 응답 파싱
                question, answer = parse_gpt_response(gpt_response)

                if question and answer:
                    all_qa_pairs.append({
                        "question": question,
                        "answer_B": answer,
                        "source_pdf": pdf_filename,
                        # "source_chunk": chunk # 필요하면 어떤 청크에서 왔는지도 기록 가능
                    })

    except Exception as e:
        print(f"❗️ {pdf_filename} 처리 중 심각한 오류 발생: {e}")
        continue

print("\n🎉 Q&A 생성 작업 완료!")
print(f"총 {len(all_qa_pairs)}개의 유효한 Q&A 쌍을 생성했습니다.")

🚀 총 233개의 PDF 파일에 대한 Q&A 생성을 시작합니다...
⚠️ 이 작업은 시간이 오래 걸릴 수 있습니다.


전체 PDF 진행률:   0%|          | 0/233 [00:00<?, ?it/s]


🎉 Q&A 생성 작업 완료!
총 1157개의 유효한 Q&A 쌍을 생성했습니다.


# [5단계] 생성된 Q&A 데이터 확인 및 파일로 저장

## 5-1. pandas DataFrame으로 변환

In [21]:
# all_qa_pairs는 딕셔너리들의 리스트인데, 이걸 표(테이블) 형태로 만들어주면
# 다루기가 훨씬 편해져. pandas의 DataFrame이 바로 그 역할을 해.
df_qa = pd.DataFrame(all_qa_pairs)

## 5-2. 결과 확인

In [22]:
# 생성된 데이터프레임의 정보를 출력해서 총 몇 개의 데이터가 있는지, 빠진 값은 없는지 확인.
print("--- 생성된 데이터셋 정보 ---")
df_qa.info()

# .head()를 사용해서 맨 위 5개 행을 샘플로 출력해본다.
# Q&A가 우리가 의도한 대로 잘 만들어졌는지 눈으로 직접 확인하는 중요한 과정이야.
print("\n--- 생성된 Q&A 샘플 (상위 5개) ---")
print(df_qa.head())

--- 생성된 데이터셋 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    1157 non-null   object
 1   answer_B    1157 non-null   object
 2   source_pdf  1157 non-null   object
dtypes: object(3)
memory usage: 27.2+ KB

--- 생성된 Q&A 샘플 (상위 5개) ---
                                            question  \
0                            CD수익률의 정의와 그 용도는 무엇인가요?   
1        CD수익률 산출 과정에서 오류가 발생했을 경우, 협회는 어떤 절차를 따르나요?   
2  비상사태 발생 시 CD수익률 산출업무 중단에 따른 금융계약의 대체금리는 어떻게 결정...   
3  KB금융그룹의 개인정보보호 정책에서 관리적, 기술적, 물리적 조치에 대해 구체적으로...   
4  KB국민은행의 채권회수 정책에서 채무자의 인간다운 삶을 보호하기 위해 어떤 원칙을 ...   

                                            answer_B         source_pdf  
0  CD수익률이란 신용평가회사로부터 AAA이상의 신용등급을 받은 시중은행이 발행한 만기...  pdf_0_label_5.pdf  
1  협회는 오류가 발견된 경우, 오류의 발생 원인에 따라 다음과 같은 절차를 따릅니다....  pdf_0_label_5.pdf  
2  비상사태 발생으로 CD수익률 산출업무가 중단되는 경우, 해당 중단기간 동안 사

## 5-3. CSV 파일로 저장

In [23]:
# 나중에 쉽게 불러오고 다른 곳에서도 사용할 수 있도록 CSV 파일로 저장하자.
# index=False는 엑셀에서 열었을 때 맨 앞에 불필요한 번호가 생기지 않도록 하는 옵션이야.
output_filename = "KB_sLLM_QA_Dataset.csv"
df_qa.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n✅ 최종 Q&A 데이터셋이 '{output_filename}' 파일로 성공적으로 저장되었습니다!")


✅ 최종 Q&A 데이터셋이 'KB_sLLM_QA_Dataset.csv' 파일로 성공적으로 저장되었습니다!


# [6단계] Hugging Face Hub에 데이터셋 업로드

## 6-1. Hugging Face 로그인 (userdata 사용)

In [24]:
# Colab의 '비밀' 탭에 저장해 둔 Hugging Face 토큰을 사용해서 로그인한다.
from huggingface_hub import login
from google.colab import userdata

try:
    # userdata.get() 함수로 'HF_TOKEN'이라는 이름의 비밀을 가져온다.
    hf_token = userdata.get('HF_TOKEN')

    # login() 함수에 토큰을 직접 전달하여 로그인한다.
    login(token=hf_token)

    print("✅ Hugging Face 로그인 성공!")

except Exception as e:
    print(f"🚨 Hugging Face 로그인 중 오류가 발생했습니다.")
    print("Colab '비밀' 탭에 'HF_TOKEN'이 올바르게 설정되었는지 확인해주세요.")
    print(f"오류: {e}")

✅ Hugging Face 로그인 성공!


## 6-2. 저장된 CSV 파일을 Dataset 객체로 로드

In [25]:
from datasets import load_dataset

# 이전에 저장한 CSV 파일의 이름을 변수로 지정.
csv_filename = "KB_sLLM_QA_Dataset.csv"

try:
    # load_dataset 함수는 CSV, JSON 등 다양한 형식의 파일을 읽어올 수 있어.
    # 'csv' 형식을 지정하고, 파일 경로를 알려주면 돼.
    # split='train'을 사용해 바로 train 세트로 지정.
    hf_dataset = load_dataset('csv', data_files=csv_filename, split='train')

    print(f"✅ '{csv_filename}' 파일을 Hugging Face Dataset 객체로 성공적으로 변환했습니다.")
    print("\n--- 로드된 데이터셋 정보 ---")
    print(hf_dataset)

except Exception as e:
    print(f"🚨 파일을 Dataset 객체로 변환하는 중 오류가 발생했습니다: {e}")

Generating train split: 0 examples [00:00, ? examples/s]

✅ 'KB_sLLM_QA_Dataset.csv' 파일을 Hugging Face Dataset 객체로 성공적으로 변환했습니다.

--- 로드된 데이터셋 정보 ---
Dataset({
    features: ['question', 'answer_B', 'source_pdf'],
    num_rows: 1157
})


## 6-3. 데이터셋을 Hub에 업로드(push)

In [26]:
try:
    hf_username = userdata.get('HF_USERNAME')
    repo_name = f"{hf_username}/KB-sLLM-QA-Dataset-Generated"

    print(f"🚀 데이터셋을 Hugging Face Hub에 업로드합니다...")
    print(f"Repository: {repo_name}")

    # .push_to_hub() 함수가 모든 작업을 알아서 해줘.
    # private=True: 데이터셋을 비공개로 설정
    # commit_message: 업로드 기록에 남길 메시지
    hf_dataset.push_to_hub(
        repo_id=repo_name,
        private=True,
        commit_message="Upload 1,157 Q&A pairs generated from 233 KB Financial Group PDFs"
    )

    print("\n🎉 데이터셋 업로드 성공!")
    print("허깅페이스 프로필에서 데이터셋을 확인해보세요.")
    print(f"URL: https://huggingface.co/datasets/{repo_name}")

except Exception as e:
    print(f"🚨 데이터셋 업로드 중 오류가 발생했습니다: {e}")

🚀 데이터셋을 Hugging Face Hub에 업로드합니다...
Repository: rucipheryn/KB-sLLM-QA-Dataset-Generated


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]


🎉 데이터셋 업로드 성공!
허깅페이스 프로필에서 데이터셋을 확인해보세요.
URL: https://huggingface.co/datasets/rucipheryn/KB-sLLM-QA-Dataset-Generated
